# Spark DataFrames Project Exercise 

#### Use the walmart_stock.csv file to Answer and complete the  tasks below!

#### Start a simple Spark Session

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("DataframeExercise").getOrCreate()

#### Load the Walmart Stock CSV File, have Spark infer the data types.

In [ ]:
walmart = spark.read.csv("../datasets/walmart_stock.csv",inferSchema=True,header=True)

#### What are the column names?

In [ ]:
walmart.columns

#### What does the Schema look like?

In [ ]:
walmart.printSchema()

#### Print out the first 5 columns.

In [ ]:
walmart.show(5)

#### Use describe() to learn about the DataFrame.

In [ ]:
walmart.describe().show()

## Bonus Question!
#### There are too many decimal places for mean and stddev in the describe() dataframe. Format the numbers to just show up to two decimal places. Pay careful attention to the datatypes that .describe() returns, we didn't cover how to do this exact formatting, but we covered something very similar. [Check this link for a hint](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Column.cast)

If you get stuck on this, don't worry, just view the solutions.

In [ ]:
walmart_stats = walmart.describe()

In [ ]:
type(walmart_stats)

In [ ]:
walmart_stats.printSchema()

In [ ]:
walmart_stats.show()

In [ ]:
from pyspark.sql.functions import format_number

In [ ]:
help(format_number)

In [ ]:
walmart_stats.select(walmart_stats["summary"],
                     format_number(walmart_stats["Open"].cast('float'),2).alias("Open"),
                     format_number(walmart_stats["High"].cast('float'),2).alias("High"),
                     format_number(walmart_stats["Low"].cast('float'),2).alias("Low"),
                     format_number(walmart_stats["Close"].cast('float'),2).alias("Close"),
                     walmart_stats["Volume"].cast('int').alias("Volume")
                    ).show()

#### Create a new dataframe with a column called HV Ratio that is the ratio of the High Price versus volume of stock traded for a day.

In [ ]:
walamrt2 = walmart.withColumn("HV Ratio", walmart["High"]/walmart["Volume"])
walamrt2.select("HV Ratio").show()

#### What day had the Peak High in Price?

In [ ]:
walmart.orderBy(walmart["High"].desc()).head(1)

In [ ]:
walmart.orderBy(walmart["High"].desc()).head(1)[0][0]

#### What is the mean of the Close column?

In [ ]:
from pyspark.sql.functions import (mean,max,min)

In [ ]:
walmart.select(mean(walmart['Close'])).show()

#### What is the max and min of the Volume column?

In [ ]:
walmart.select([max(walmart['Volume']),min('Volume')]).show()

#### How many days was the Close lower than 60 dollars?

In [ ]:
walmart.select(walmart['Close']<60).show()

In [ ]:
from pyspark.sql.functions import count

In [ ]:
result = walmart.filter(walmart['Close'] < 60)
result.select(count(walmart['Close']).alias("Count < 60")).show()

#### What percentage of the time was the High greater than 80 dollars ?
#### In other words, (Number of Days High>80)/(Total Days in the dataset)

In [ ]:
((walmart.filter(walmart['High']>80)).count() * 1.0 / walmart.count())*100

#### What is the Pearson correlation between High and Volume?
#### [Hint](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrameStatFunctions.corr)

In [ ]:
from pyspark.sql.functions import corr

In [ ]:
help(corr)

In [ ]:
walmart.select(corr('High','Volume')).show()

In [ ]:
from pyspark.ml.stat import Correlation

In [ ]:
help(Correlation)

#### What is the max High per year?

In [ ]:
from pyspark.sql.functions import year

In [ ]:
walmart3 = walmart.withColumn("Year", year(walmart['Date']))

In [ ]:
walmart3.show()

In [ ]:
max_year = walmart3.groupBy('Year').max()

In [ ]:
max_year.show()

In [ ]:
max_year.select('Year','max(High)').show()

#### What is the average Close for each Calendar Month?
#### In other words, across all the years, what is the average Close price for Jan,Feb, Mar, etc... Your result will have a value for each of these months. 

In [ ]:
from pyspark.sql.functions import month,avg

In [ ]:
walmart4 = walmart.withColumn("Month", month(walmart['Date']))

In [ ]:
avg_month = walmart4.groupBy('Month').avg()

In [ ]:
avg_month.select('Month','avg(Close)').show()